# Combining Datasets in Pandas: Concat and Append

- Combining datasets allows for deeper analysis by integrating multiple data sources.

- Operations can range from simple concatenations to complex database-style joins and merges.

- Pandas provides functions that make these operations efficient and easy to perform.

# 1. pd.concat

In [3]:
import pandas as pd 
import numpy as np

In [9]:
# Create a function to quickly generate a DF

from typing import List, Iterable, Any
def make_df(cols: Iterable[Any], rows:Iterable[Any]) -> pd.DataFrame:
    """
    Quickly make a Dataframe
    """

    data = {c: [str(c)+str(i) for i in rows]
            for c in cols}
    return pd.DataFrame(data, index=rows)

In [10]:
make_df('ABC',range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2
